In [68]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
import arcpy
import geopandas as gpd

bRedownload = False
bExport = True

In [54]:
dfFlags = pd.read_json('_site/data/flags.json', orient='records')
dfFlags

,flagName,flagDescription,flagCriteria
0,FL_REV,Not reviewed,([NOTES]=='') & ([NOTES_FURREV]=='')
1,FL_LTPRV,Less than previous forecast year,(([MF2023]+[ADJ2023])<([M2019]+[aadtAdjFactor]...
2,FL_ZERO,Zero volume,(([MF2023]+[ADJ2023])<=0) | (([MF2028]+[ADJ202...
3,FL_HIADJ,Large model adjustment factor,((([aadtAdjFactor]/([M2019]+[aadtAdjFactor]))>...
4,FL_50LT19,HPMS: 2050 < 2019,(([MF2050]+[ADJ2050])<([M2019]+[aadtAdjFactor]))
5,FL_503X19,HPMS: 2050 > 3 x 2019,(([MF2050]+[ADJ2050])>(3*([M2019]+[aadtAdjFact...
6,FL_FURREV,Further Review,([NOTES_FURREV]!='')
7,FL_SEG,Segment Note,([NOTES_SEG]!='')


In [55]:
if bRedownload==True:

    # Define the URL for querying the feature service
    url = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Forecasts_gdb/FeatureServer/0/query"

    # Initialize the list to store all features
    all_features = []

    # Base parameters for the request
    params = {
        "f": "geojson",
        "where": "1=1",
        "outFields": "*",
        "returnGeometry": "true",
        "resultRecordCount": 1000  # Adjust based on server limit
    }

    # Initialize the offset
    offset = 0

    while True:
        # Update the offset for each batch
        params['resultOffset'] = offset
        
        # Send the request
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Convert the response to GeoJSON
            data_geojson = response.json()
            
            # Check if no features were returned, which means we've fetched all records
            if not data_geojson['features']:
                break
            
            # Add the fetched features to the list
            all_features.extend(data_geojson['features'])
            
            # Increase the offset for the next batch
            offset += params['resultRecordCount']
        else:
            print(f"Error fetching data: {response.status_code}")
            break

    # Convert all fetched features into a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(all_features)

    display(gdf)


,geometry,OBJECTID,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFactor,DYVOL2019,...,FL_REV,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME,Shape__Length
0,"LINESTRING (-114.04890 39.05713, -114.03657 39...",1,0006_000.0,0,0.9840,1.0276,1.0316,1.02960,286,126.5,...,0,1,0,0,1,0,1,0,Millard,1379.206909
1,"LINESTRING (-114.03657 39.05618, -114.02603 39...",2,0006_000.7,0,0.9840,1.0276,1.0316,1.02960,198,213.6,...,0,1,0,0,1,0,1,0,Millard,31801.743094
2,"LINESTRING (-113.75753 39.05992, -113.75747 39...",3,0006_016.0,0,0.9840,1.0276,1.0316,1.02960,337,76.9,...,0,1,0,0,1,0,1,0,Millard,62131.802505
3,"LINESTRING (-113.24932 39.05910, -113.24471 39...",4,0006_046.0,0,0.9840,1.0276,1.0316,1.02960,294,76.9,...,0,1,0,0,1,0,1,0,Millard,29438.458712
4,"LINESTRING (-113.03027 39.17300, -113.02993 39...",5,0006_060.2,0,0.9840,1.0276,1.0316,1.02960,290,80.3,...,0,1,0,0,1,0,1,0,Millard,36000.972827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8962,"LINESTRING (-112.09801 40.68196, -112.10016 40...",8963,WFRC_8469,1,0.0000,0.0000,0.0000,0.00000,0,0.0,...,0,0,1,0,0,0,0,0,Salt Lake,1206.458573
8963,"LINESTRING (-112.09189 40.67653, -112.09801 40...",8964,WFRC_8470,1,0.0000,0.0000,0.0000,0.00000,0,0.0,...,0,0,1,0,0,0,0,0,Salt Lake,1047.665090
8964,"LINESTRING (-111.99419 40.48090, -111.99394 40...",8965,WFRC_8471,1,1.0924,1.0104,1.0243,1.01735,0,0.0,...,0,0,1,0,0,0,0,0,Salt Lake,858.554874
8965,"LINESTRING (-111.89397 40.55138, -111.89402 40...",8966,WFRC_8472,1,0.0000,0.0000,0.0000,0.00000,0,0.0,...,0,0,1,0,0,0,0,0,Salt Lake,536.957257


In [56]:
from datetime import datetime
from pathlib import Path
import geopandas as gpd

# Assuming gdf is your GeoDataFrame
# gdf = ...

# Generate a timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Define the path with the timestamp using Path object
file_path = Path(f'backup/forecastsegments_backup_{timestamp}.shp')

# Create the 'intermediate' directory if it does not exist
file_path.parent.mkdir(parents=True, exist_ok=True)

# Export the GeoDataFrame to a Shapefile
gdf.to_file(file_path)

print(f"GeoDataFrame exported to {file_path}")


<ipython-input-56-3713e2637c58>:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(file_path)


GeoDataFrame exported to backup\forecastsegments_backup_20240301_112138.shp


In [57]:
### IMPORTANT... USING HARD-CODED DATESTAMP SINCE I WANT TO MAKE SURE IT'S THE RIGHT FILE IN A POINT OF TIME, AND NOT FROM A SUBSEQUENT AUTO-GENERATED FILE

# Define the path of the existing shapefile
existing_shapefile_path = 'backup/forecastsegments_backup_20240301_110136.shp'

# Define the new path where you want to save the shapefile
# If you want to include a dynamic timestamp, you can generate it as shown previously
new_shapefile_path = 'intermediate/forecastsegments_backup_20240301_110136_priorToCacheReset.shp'  # Adjust as needed

# Read the shapefile into a GeoDataFrame
gdf = gpd.read_file(existing_shapefile_path)

# Optionally, make any modifications to the GeoDataFrame here

# Save the GeoDataFrame to a new shapefile
gdf.to_file(new_shapefile_path)

print(f"Shapefile saved as {new_shapefile_path}")

Shapefile saved as intermediate/forecastsegments_backup_20240301_110136_priorToCacheReset.shp


In [120]:
gdf.rename(columns={'aadtAdjFac':"aadtAdjFactor","NOTES_FURR":"NOTES_FURREV"}, inplace=True)

gdfCache = gdf[(gdf['SUBAREAID']==2)].copy()
display(gdfCache)
gdfNonCache = gdf[(gdf['SUBAREAID']!=2)].copy()
display(gdfNonCache)

,OBJECTID,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFactor,DYVOL2019,DYVOL2023,...,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME,Shape__Len,geometry
454,455,0023_000.0,2,1.0918,1.0107,1.0324,1.02155,122,2016.4,2018.0,...,0,0,0,0,0,1,1,Cache,3421.450232,"LINESTRING (-111.93343 41.61601, -111.93366 41..."
455,456,0023_001.6,2,1.0918,1.0107,1.0324,1.02155,1729,1338.7,1325.7,...,1,0,0,0,0,0,1,Cache,1252.488988,"LINESTRING (-111.93436 41.63879, -111.93433 41..."
456,457,0023_002.1,2,1.0918,1.0107,1.0324,1.02155,521,2657.7,2630.5,...,0,0,0,0,0,0,1,Cache,828.269980,"LINESTRING (-111.93407 41.64720, -111.93405 41..."
457,458,0023_002.5,2,0.9840,1.0276,1.0316,1.02960,514,1668.1,1643.4,...,1,0,0,0,0,0,1,Cache,3524.130854,"LINESTRING (-111.93923 41.64965, -111.93931 41..."
458,459,0023_004.2,2,0.9840,1.0276,1.0316,1.02960,704,1480.8,1467.4,...,1,0,0,0,0,0,1,Cache,4564.742972,"LINESTRING (-111.95372 41.67068, -111.95372 41..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7905,7906,Cache_4057,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,0,1,0,0,0,1,1,Cache,246.775681,"LINESTRING (-111.83337 41.67465, -111.83115 41..."
7906,7907,Cache_4058,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,0,1,0,0,0,1,1,Cache,258.269346,"LINESTRING (-111.81820 41.67628, -111.81818 41..."
7907,7908,Cache_4059,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,0,1,0,0,0,1,1,Cache,1420.637310,"LINESTRING (-111.88316 41.67533, -111.88332 41..."
7908,7909,Cache_4060,2,1.0918,1.0107,1.0324,1.02155,0,0.0,0.0,...,0,1,0,0,0,1,1,Cache,2581.583996,"LINESTRING (-111.88173 41.70703, -111.87616 41..."


,OBJECTID,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFactor,DYVOL2019,DYVOL2023,...,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME,Shape__Len,geometry
0,1,0006_000.0,0,0.9840,1.0276,1.0316,1.02960,286,126.5,99.7,...,1,0,0,1,0,1,0,Millard,1379.206909,"LINESTRING (-114.04890 39.05713, -114.03657 39..."
1,2,0006_000.7,0,0.9840,1.0276,1.0316,1.02960,198,213.6,216.5,...,1,0,0,1,0,1,0,Millard,31801.743094,"LINESTRING (-114.03657 39.05618, -114.02603 39..."
2,3,0006_016.0,0,0.9840,1.0276,1.0316,1.02960,337,76.9,73.9,...,1,0,0,1,0,1,0,Millard,62131.802505,"LINESTRING (-113.75753 39.05992, -113.75747 39..."
3,4,0006_046.0,0,0.9840,1.0276,1.0316,1.02960,294,76.9,73.9,...,1,0,0,1,0,1,0,Millard,29438.458712,"LINESTRING (-113.24932 39.05910, -113.24471 39..."
4,5,0006_060.2,0,0.9840,1.0276,1.0316,1.02960,290,80.3,77.4,...,1,0,0,1,0,1,0,Millard,36000.972827,"LINESTRING (-113.03027 39.17300, -113.02993 39..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8962,8963,WFRC_8469,1,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,0,1,0,0,0,0,0,Salt Lake,1206.458573,"LINESTRING (-112.09801 40.68196, -112.10016 40..."
8963,8964,WFRC_8470,1,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,0,1,0,0,0,0,0,Salt Lake,1047.665090,"LINESTRING (-112.09189 40.67653, -112.09801 40..."
8964,8965,WFRC_8471,1,1.0924,1.0104,1.0243,1.01735,0,0.0,0.0,...,0,1,0,0,0,0,0,Salt Lake,858.554874,"LINESTRING (-111.99419 40.48090, -111.99394 40..."
8965,8966,WFRC_8472,1,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,0,1,0,0,0,0,0,Salt Lake,536.957257,"LINESTRING (-111.89397 40.55138, -111.89402 40..."


In [121]:
dfFlags

,flagName,flagDescription,flagCriteria
0,FL_REV,Not reviewed,([NOTES]=='') & ([NOTES_FURREV]=='')
1,FL_LTPRV,Less than previous forecast year,(([MF2023]+[ADJ2023])<([M2019]+[aadtAdjFactor]...
2,FL_ZERO,Zero volume,(([MF2023]+[ADJ2023])<=0) | (([MF2028]+[ADJ202...
3,FL_HIADJ,Large model adjustment factor,((([aadtAdjFactor]/([M2019]+[aadtAdjFactor]))>...
4,FL_50LT19,HPMS: 2050 < 2019,(([MF2050]+[ADJ2050])<([M2019]+[aadtAdjFactor]))
5,FL_503X19,HPMS: 2050 > 3 x 2019,(([MF2050]+[ADJ2050])>(3*([M2019]+[aadtAdjFact...
6,FL_FURREV,Further Review,([NOTES_FURREV]!='')
7,FL_SEG,Segment Note,([NOTES_SEG]!='')


In [122]:
gdfCache.columns

Index(['OBJECTID', 'SEGID', 'SUBAREAID', 'FAC_WDAVG', 'FAC_SPR', 'FAC_FAL',
       'FAC_SPRFAL', 'aadtAdjFactor', 'DYVOL2019', 'DYVOL2023', 'DYVOL2028',
       'DYVOL2032', 'DYVOL2042', 'DYVOL2050', 'M2019', 'M2023', 'M2028',
       'M2032', 'M2042', 'M2050', 'MF2019', 'MF2023', 'MF2028', 'MF2032',
       'MF2042', 'MF2050', 'CO_FIPS', 'PLANAREA', 'F_AREA', 'ADJHIST',
       'ADJ2019', 'ADJ2023', 'ADJ2028', 'ADJ2032', 'ADJ2042', 'ADJ2050',
       'OV_REV', 'OV_LTPRV', 'OV_ZERO', 'OV_HIADJ', 'OV_50LT19', 'OV_503X19',
       'OV_FURREV', 'NOTES', 'NOTES_FURREV', 'OV_SEG', 'NOTES_SEG', 'FL_REV',
       'FL_LTPRV', 'FL_ZERO', 'FL_HIADJ', 'FL_50LT19', 'FL_503X19',
       'FL_FURREV', 'FL_SEG', 'CO_NAME', 'Shape__Len', 'geometry'],
      dtype='object')

In [123]:
gdfCache['ADJHIST'     ] = 0
gdfCache['ADJ2023'     ] = 0
gdfCache['ADJ2028'     ] = 0
gdfCache['ADJ2032'     ] = 0
gdfCache['ADJ2042'     ] = 0
gdfCache['ADJ2050'     ] = 0
gdfCache['NOTES'       ] = ""
gdfCache['NOTES_FURREV'] = ""
gdfCache['NOTES_SEG'   ] = ""
gdfCache

,OBJECTID,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFactor,DYVOL2019,DYVOL2023,...,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME,Shape__Len,geometry
454,455,0023_000.0,2,1.0918,1.0107,1.0324,1.02155,122,2016.4,2018.0,...,0,0,0,0,0,1,1,Cache,3421.450232,"LINESTRING (-111.93343 41.61601, -111.93366 41..."
455,456,0023_001.6,2,1.0918,1.0107,1.0324,1.02155,1729,1338.7,1325.7,...,1,0,0,0,0,0,1,Cache,1252.488988,"LINESTRING (-111.93436 41.63879, -111.93433 41..."
456,457,0023_002.1,2,1.0918,1.0107,1.0324,1.02155,521,2657.7,2630.5,...,0,0,0,0,0,0,1,Cache,828.269980,"LINESTRING (-111.93407 41.64720, -111.93405 41..."
457,458,0023_002.5,2,0.9840,1.0276,1.0316,1.02960,514,1668.1,1643.4,...,1,0,0,0,0,0,1,Cache,3524.130854,"LINESTRING (-111.93923 41.64965, -111.93931 41..."
458,459,0023_004.2,2,0.9840,1.0276,1.0316,1.02960,704,1480.8,1467.4,...,1,0,0,0,0,0,1,Cache,4564.742972,"LINESTRING (-111.95372 41.67068, -111.95372 41..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7905,7906,Cache_4057,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,0,1,0,0,0,1,1,Cache,246.775681,"LINESTRING (-111.83337 41.67465, -111.83115 41..."
7906,7907,Cache_4058,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,0,1,0,0,0,1,1,Cache,258.269346,"LINESTRING (-111.81820 41.67628, -111.81818 41..."
7907,7908,Cache_4059,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,0,1,0,0,0,1,1,Cache,1420.637310,"LINESTRING (-111.88316 41.67533, -111.88332 41..."
7908,7909,Cache_4060,2,1.0918,1.0107,1.0324,1.02155,0,0.0,0.0,...,0,1,0,0,0,1,1,Cache,2581.583996,"LINESTRING (-111.88173 41.70703, -111.87616 41..."


In [124]:
# Set flags to zero and OVERRIDES to zero!

# Assuming dfFlags['flagName'] contains the column names
# And gdfCache is your GeoDataFrame

# Iterate through each flag name in the dfFlags['flagName'] series
for column_name in dfFlags['flagName']:
    # Check if the column exists in gdfCache to avoid KeyError
    if column_name in gdfCache.columns:
        # Set the entire column to 0
        gdfCache[column_name] = 0
        gdfCache[column_name.replace('FL_','OV_')] = 0

In [125]:
gdfCache.columns

Index(['OBJECTID', 'SEGID', 'SUBAREAID', 'FAC_WDAVG', 'FAC_SPR', 'FAC_FAL',
       'FAC_SPRFAL', 'aadtAdjFactor', 'DYVOL2019', 'DYVOL2023', 'DYVOL2028',
       'DYVOL2032', 'DYVOL2042', 'DYVOL2050', 'M2019', 'M2023', 'M2028',
       'M2032', 'M2042', 'M2050', 'MF2019', 'MF2023', 'MF2028', 'MF2032',
       'MF2042', 'MF2050', 'CO_FIPS', 'PLANAREA', 'F_AREA', 'ADJHIST',
       'ADJ2019', 'ADJ2023', 'ADJ2028', 'ADJ2032', 'ADJ2042', 'ADJ2050',
       'OV_REV', 'OV_LTPRV', 'OV_ZERO', 'OV_HIADJ', 'OV_50LT19', 'OV_503X19',
       'OV_FURREV', 'NOTES', 'NOTES_FURREV', 'OV_SEG', 'NOTES_SEG', 'FL_REV',
       'FL_LTPRV', 'FL_ZERO', 'FL_HIADJ', 'FL_50LT19', 'FL_503X19',
       'FL_FURREV', 'FL_SEG', 'CO_NAME', 'Shape__Len', 'geometry'],
      dtype='object')

In [126]:
# add flag columns
# Loop through the rows of 'dfFlags'
for index, row in dfFlags.iterrows():
    # Get the flag name from the 'flagName' column
    flag_name = row['flagName']

    # Get the flag criteria from the 'flagCriteria' column (if needed)
    flag_criteria = row['flagCriteria']

    # Add a new column to 'dfSegs' with the flag name and set it equal to evaluated criteria, prepare criteria string with replace
    expression = "gdfCache['" + flag_name + "'] = " + flag_criteria.replace("[", "gdfCache['").replace("]", "']")
    print(expression)

    # execute expression!
    exec(expression)
  

gdfCache.fillna(0,inplace=True)
gdfCache

gdfCache['FL_REV'] = (gdfCache['NOTES']=='') & (gdfCache['NOTES_FURREV']=='')
gdfCache['FL_LTPRV'] = ((gdfCache['MF2023']+gdfCache['ADJ2023'])<(gdfCache['M2019']+gdfCache['aadtAdjFactor'])) | ((gdfCache['MF2028']+gdfCache['ADJ2028'])<(gdfCache['MF2023']+gdfCache['ADJ2023'])) | ((gdfCache['MF2032']+gdfCache['ADJ2032'])<(gdfCache['MF2028']+gdfCache['ADJ2028'])) | ((gdfCache['MF2042']+gdfCache['ADJ2042'])<(gdfCache['MF2032']+gdfCache['ADJ2032'])) | ((gdfCache['MF2050']+gdfCache['ADJ2050'])<(gdfCache['MF2042']+gdfCache['ADJ2042']))
gdfCache['FL_ZERO'] = ((gdfCache['MF2023']+gdfCache['ADJ2023'])<=0) | ((gdfCache['MF2028']+gdfCache['ADJ2028'])<=0) | ((gdfCache['MF2032']+gdfCache['ADJ2032'])<=0) | ((gdfCache['MF2042']+gdfCache['ADJ2042'])<=0) | ((gdfCache['MF2050']+gdfCache['ADJ2050'])<=0)
gdfCache['FL_HIADJ'] = (((gdfCache['aadtAdjFactor']/(gdfCache['M2019']+gdfCache['aadtAdjFactor']))>0.95) | ((gdfCache['aadtAdjFactor']/(gdfCache['M2019']+gdfCache['aadtAdjFactor']))<-0.95))
gdfCache['FL_50L

,OBJECTID,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFactor,DYVOL2019,DYVOL2023,...,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME,Shape__Len,geometry
454,455,0023_000.0,2,1.0918,1.0107,1.0324,1.02155,122,2016.4,2018.0,...,False,False,False,False,False,False,False,Cache,3421.450232,"LINESTRING (-111.93343 41.61601, -111.93366 41..."
455,456,0023_001.6,2,1.0918,1.0107,1.0324,1.02155,1729,1338.7,1325.7,...,True,False,False,False,False,False,False,Cache,1252.488988,"LINESTRING (-111.93436 41.63879, -111.93433 41..."
456,457,0023_002.1,2,1.0918,1.0107,1.0324,1.02155,521,2657.7,2630.5,...,True,False,False,False,False,False,False,Cache,828.269980,"LINESTRING (-111.93407 41.64720, -111.93405 41..."
457,458,0023_002.5,2,0.9840,1.0276,1.0316,1.02960,514,1668.1,1643.4,...,True,False,False,False,False,False,False,Cache,3524.130854,"LINESTRING (-111.93923 41.64965, -111.93931 41..."
458,459,0023_004.2,2,0.9840,1.0276,1.0316,1.02960,704,1480.8,1467.4,...,True,False,False,False,False,False,False,Cache,4564.742972,"LINESTRING (-111.95372 41.67068, -111.95372 41..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7905,7906,Cache_4057,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,False,True,False,False,False,False,False,Cache,246.775681,"LINESTRING (-111.83337 41.67465, -111.83115 41..."
7906,7907,Cache_4058,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,False,True,False,False,False,False,False,Cache,258.269346,"LINESTRING (-111.81820 41.67628, -111.81818 41..."
7907,7908,Cache_4059,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,False,True,False,False,False,False,False,Cache,1420.637310,"LINESTRING (-111.88316 41.67533, -111.88332 41..."
7908,7909,Cache_4060,2,1.0918,1.0107,1.0324,1.02155,0,0.0,0.0,...,False,True,False,False,True,False,False,Cache,2581.583996,"LINESTRING (-111.88173 41.70703, -111.87616 41..."


In [127]:
gdfCache[dfFlags['flagName'].tolist()].sum()

FL_REV       489
FL_LTPRV     360
FL_ZERO       41
FL_HIADJ      57
FL_50LT19     22
FL_503X19     69
FL_FURREV      0
FL_SEG         0
dtype: int64

In [128]:
gdfNew = pd.concat([gdfNonCache,gdfCache])
gdfNew

,OBJECTID,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFactor,DYVOL2019,DYVOL2023,...,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME,Shape__Len,geometry
0,1,0006_000.0,0,0.9840,1.0276,1.0316,1.02960,286,126.5,99.7,...,1,0,0,1,0,1,0,Millard,1379.206909,"LINESTRING (-114.04890 39.05713, -114.03657 39..."
1,2,0006_000.7,0,0.9840,1.0276,1.0316,1.02960,198,213.6,216.5,...,1,0,0,1,0,1,0,Millard,31801.743094,"LINESTRING (-114.03657 39.05618, -114.02603 39..."
2,3,0006_016.0,0,0.9840,1.0276,1.0316,1.02960,337,76.9,73.9,...,1,0,0,1,0,1,0,Millard,62131.802505,"LINESTRING (-113.75753 39.05992, -113.75747 39..."
3,4,0006_046.0,0,0.9840,1.0276,1.0316,1.02960,294,76.9,73.9,...,1,0,0,1,0,1,0,Millard,29438.458712,"LINESTRING (-113.24932 39.05910, -113.24471 39..."
4,5,0006_060.2,0,0.9840,1.0276,1.0316,1.02960,290,80.3,77.4,...,1,0,0,1,0,1,0,Millard,36000.972827,"LINESTRING (-113.03027 39.17300, -113.02993 39..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7905,7906,Cache_4057,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,False,True,False,False,False,False,False,Cache,246.775681,"LINESTRING (-111.83337 41.67465, -111.83115 41..."
7906,7907,Cache_4058,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,False,True,False,False,False,False,False,Cache,258.269346,"LINESTRING (-111.81820 41.67628, -111.81818 41..."
7907,7908,Cache_4059,2,0.0000,0.0000,0.0000,0.00000,0,0.0,0.0,...,False,True,False,False,False,False,False,Cache,1420.637310,"LINESTRING (-111.88316 41.67533, -111.88332 41..."
7908,7909,Cache_4060,2,1.0918,1.0107,1.0324,1.02155,0,0.0,0.0,...,False,True,False,False,True,False,False,Cache,2581.583996,"LINESTRING (-111.88173 41.70703, -111.87616 41..."


In [129]:
print (gdfNew.crs)
gdfNew.crs = 'EPSG:4326'  # EPSG:4326 represents WGS84 Latitude/Longitude
print (gdfNew.crs)

EPSG:4326
EPSG:4326


In [130]:
# Generate a timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Define the path with the timestamp using Path object
file_path = Path(f'results/ForecastSegments_{timestamp}.shp')

# Create the 'intermediate' directory if it does not exist
file_path.parent.mkdir(parents=True, exist_ok=True)

# Export the GeoDataFrame to a Shapefile
gdfNew.to_file(file_path)

print(f"GeoDataFrame exported to {file_path}")


<ipython-input-130-b33c35a28269>:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdfNew.to_file(file_path)


GeoDataFrame exported to results\ForecastSegments_20240301_120628.shp


In [132]:
# export
#if bShapefileExport: #gdfNew.spatial.to_featureclass('results/ForecastSegments/ForecastSegments.shp',sanitize_columns=False)
from arcgis.features import GeoAccessor

##########################################################################################
###  IMPORTANT HARD CODED HERE!!!!! IF RERUN NEED TO FIX
##########################################################################################
shpFile = 'results\ForecastSegments_20240301_120628.shp'


if bExport:
    # Define the geodatabase path
    gdb_path        = r"results\Forecasts.gdb"
    gdb_path_backup = r"backup\Forecasts_backup.gdb"

    # Check if the geodatabase exists, if not, create it
    if not arcpy.Exists(gdb_path):
        arcpy.CreateFileGDB_management(r"results", "Forecasts.gdb")
        
    # Check if the geodatabase exists, if not, create it
    if not arcpy.Exists(gdb_path_backup):
        arcpy.CreateFileGDB_management(r"backup", "Forecasts_backup.gdb")

    # Export SDF to the geodatabase as a feature class
    feature_class_name = "ForecastSegments"

    # Generate a filename with the current timestamp in the specified folder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    feature_class_name_with_timestamp = f"ForecastSegments_{timestamp}"

    # file to replace
    arcpy.FeatureClassToFeatureClass_conversion(shpFile, gdb_path_backup, feature_class_name_with_timestamp)
    print(f"Feature class {feature_class_name_with_timestamp} created in {gdb_path_backup}")

    # backup file
    arcpy.FeatureClassToFeatureClass_conversion(shpFile, gdb_path, feature_class_name)
    print(f"Feature class {feature_class_name} created in {gdb_path}")

else:
    print ('not exported')


Feature class ForecastSegments_20240301_120653 created in backup\Forecasts_backup.gdb
Feature class ForecastSegments created in results\Forecasts.gdb
